In [6]:

import os

In [5]:
os.chdir('e:\\sonal\\Nmims\\Sem 3\\Mlops\\MLOPs_indvidual')

In [10]:
%pwd

'e:\\sonal\\Nmims\\Sem 3\\Mlops\\MLOPs_indvidual'

In [ ]:
import os
import mlflow

os.environ['MLFLOW_TRACKING_USERNAME'] = 'SonalGupta21'
os.environ['MLFLOW_TRACKING_PASSWORD'] = 'd3cf9244c2c33daf330b4eae478cae68199391e1'
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/SonalGupta21/MLOPs_individual.mlflow'





In [1]:
import dagshub
dagshub.init(repo_owner='sonalgupta21',
             repo_name='MLOPs_indvidual',
             mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)



❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=8140cde9-06a0-4f91-a99c-095f143eab5e&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=b96e6dc2954638d40f02083d3eb2c1b96ca3711fdb502e5f7f7d4307453aa06d




Accessing as SonalGupta21

Initialized MLflow to track repo "sonalgupta21/MLOPs_indvidual"

Repository sonalgupta21/MLOPs_indvidual initialized!

🏃 View run orderly-mole-23 at: https://dagshub.com/sonalgupta21/MLOPs_indvidual.mlflow/#/experiments/0/runs/bd626abcf500402b889472d56754226b
🧪 View experiment at: https://dagshub.com/sonalgupta21/MLOPs_indvidual.mlflow/#/experiments/0


In [8]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [9]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri=os.getenv("MLFLOW_TRACKING_URI"),
           
        )

        return model_evaluation_config


In [11]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [25]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    


    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Save model locally and log it as an artifact (Dagshub-friendly way)
            joblib.dump(model, "model.joblib")
            mlflow.log_artifact("model.joblib", artifact_path="model")



    


In [26]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-11-01 11:08:36,970: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-11-01 11:08:36,972: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-01 11:08:36,977: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-11-01 11:08:36,979: INFO: common: created directory at: artifacts]
[2025-11-01 11:08:36,980: INFO: common: created directory at: artifacts/model_evaluation]
[2025-11-01 11:08:38,126: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
🏃 View run industrious-fowl-481 at: https://dagshub.com/sonalgupta21/MLOPs_indvidual.mlflow/#/experiments/0/runs/b1fcc4fb2d1941b896c4113eb7787fc8
🧪 View experiment at: https://dagshub.com/sonalgupta21/MLOPs_indvidual.mlflow/#/experiments/0
